In [62]:
from selenium import webdriver
from bs4 import BeautifulSoup
from collections import Counter

import pandas as pd 
import re
import time

import matplotlib.pyplot as plt
import seaborn as sns 

import matplotlib
from matplotlib import font_manager, rc
import platform
import warnings

In [63]:
driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://unipass.customs.go.kr/ets/index.do?menuId=ETS_MNU_00000107")
# driver.get("https://www.jobplanet.co.kr/search?query=%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D&category=search_new&search_keyword_hint_id=6&_rs_con=seach&_rs_act=keyword_search")
time.sleep(2)

In [64]:
# search_form=driver.find_element_by_id("TRS0104012Q_fmSearch")

In [65]:
# getNitemtradeList
code_df = pd.read_excel("../data/관세청조회코드_v2.7.xlsx", sheet_name=None)
items_df =code_df['품목별 코드']
country_df = code_df['국가별코드']
sinseong_df =code_df['신성질별코드']
export_df = code_df['수출 수입코드']

In [66]:
# *대상품목(16): 배추, 무, 양파, 건고추, 마늘, 대파, 얼갈이배추, 양배추, 깻잎, 시금치, 미나리, 당근, 파프리카, 새송이, 팽이버섯, 토마토
# *대상품종(5): 청상추, 백다다기, 애호박, 캠벨얼리, 샤인마스캇

list_PUM = ['배추', '무', '양파', '건고추', '마늘', '대파', '얼갈이배추', '양배추', '깻잎', '시금치', '미나리', '당근', '파프리카', '새송이', '팽이버섯', '토마토']
list_KIN = ['청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇']

In [67]:
PUM_code_df = items_df[items_df['품목명'].isin(list_PUM)]
len(PUM_code_df)
# PUM_code_df
# items_df[items_df['품목명'].isin(list_KIN)]

print(PUM_code_df['품목명'].unique())
# PUM_code_df

['양파' '마늘' '양배추' '배추' '당근' '무' '팽이버섯' '토마토']


In [68]:
# 품목코드 문자열로 타입 변경
PUM_code_df = PUM_code_df.astype({'품목코드':'str'})

# PUM_code_df['품목코드'][PUM_code_df['품목코드'].str.len() == 9].str.zfill(width=10)
# 품목코드 10자리, 6자리로 변경 앞에 0 을 붙임.
PUM_code_df['품목코드'] = PUM_code_df['품목코드'].apply(lambda x: x.zfill(10) if (len(x) == 9) else x )
PUM_code_df['품목코드'] = PUM_code_df['품목코드'].apply(lambda x: x.zfill(6) if (len(x) == 5) else x )

In [69]:
PUM_code_df

,품목코드,품목명
952,0703101000,양파
954,070320,마늘
964,0704901000,양배추
965,0704902000,배추
976,0706101000,당근
979,0706901000,무
1004,0709595000,팽이버섯
1025,0710801000,양파
1026,0710802000,마늘
1028,0710804000,당근


In [70]:
def search_period_set(arg_driver, arg_pstart, arg_pend):
    search_form=arg_driver.find_element_by_id("TRS0104012Q_fmSearch")

    # 조회기간3년조회 가능 
    priodKind=search_form.find_element_by_id("TRS0104012Q_priodKind")    
    # 조회기간, 월 단위
    priodKind.send_keys("월")    

    # 시작 년월
    priodFr=search_form.find_element_by_id("TRS0104012Q_priodFr")
    # priodFr.send_keys("2000.01")
    priodFr.send_keys(arg_pstart)

    # 종료 년월
    priodTo=search_form.find_element_by_id("TRS0104012Q_priodTo")
    # priodTo.send_keys("2003.12")
    priodTo.send_keys(arg_pend)
    return

def serch_pum_set(arg_driver, arg_pumcode):
    search_form=arg_driver.find_element_by_id("TRS0104012Q_fmSearch")
    # 품목코드 
    hsSgn02=search_form.find_element_by_id("TRS0104012Q_hsSgn02")
    hsSgn04=search_form.find_element_by_id("TRS0104012Q_hsSgn04")
    hsSgn06=search_form.find_element_by_id("TRS0104012Q_hsSgn06")
    hsSgn10=search_form.find_element_by_id("TRS0104012Q_hsSgn10")
    hsSgn02.clear()
    hsSgn04.clear()
    hsSgn06.clear()
    hsSgn10.clear()

    # 품목코드가 6자리 이상이면 (10자리)
    if len(arg_pumcode) > 6:
        hsSgn02.send_keys(arg_pumcode[0:2]) # 코드 앞자리 0 ~ 2 자리        
        hsSgn04.send_keys(arg_pumcode[2:4]) # 코드 앞자리 3 ~ 4 자리    
        hsSgn06.send_keys(arg_pumcode[4:6]) # 코드 앞자리 5 ~ 6 자리
        hsSgn10.send_keys(arg_pumcode[6:]) # 코드 앞자리 7 ~ 10 자리
    else: 
        hsSgn02.send_keys(arg_pumcode[0:2]) # 코드 앞자리 0 ~ 2 자리        
        hsSgn04.send_keys(arg_pumcode[2:4]) # 코드 앞자리 3 ~ 4 자리    
        hsSgn06.send_keys(arg_pumcode[4:6]) # 코드 앞자리 5 ~ 6 자리

    # 국가명 
    cntyCd=search_form.find_element_by_id("TRS0104012Q_cntyCd")
    cntyCd.send_keys("전체")
    return

def form_search(arg_driver):    
    # 조회버튼 입력 
    arg_driver.find_element_by_css_selector('#TRS0104012Q_fmSearch > div > footer > button').click()

def get_excel(arg_driver):
    # 엑셀 다운로드
    arg_driver.find_element_by_css_selector('#TRS0104012Q_downExcel_double').click()
    #TRS0104012Q_downExcel_double
    return

In [79]:
pum_code_list = list(PUM_code_df['품목코드'])
pum_code_list2 = pum_code_list[13:]
pum_code_list2

['0712391050',
 '0712901000',
 '0712902020',
 '0712902040',
 '0712902060',
 '2001909020',
 '2001909060',
 '2001909070']

In [80]:
# def search_period_set(arg_driver, arg_pstart, arg_pend, arg_pumcode):
# def serch_pum_set(arg_driver, arg_pumcode):    
# def form_search(arg_driver):    
# def get_excel(driver):

# 년 월 
year_list = list(range(2000, 2022, 3))
# year_list
month_list = ["01", "12"]
# year_list

# for pum in pum_code_list:
for pum in pum_code_list2:
    for cnt in range(len(year_list)-1):
        try: 
            result = driver.switch_to_alert()
            print(result.text)

            result.accept()
            result.dismiss()
        except:
            "arlert 창 처리"
            if cnt > 0:
                # print(year_list[cnt]+1, year_list[cnt+1], cnt)
                start_period = str(year_list[cnt]+1) + "." + month_list[0]
                end_period   = str(year_list[cnt+1]) + "." + month_list[1]
                print(start_period, end_period)
            else:
                serch_pum_set(driver, pum)
                # print(year_list[cnt], year_list[cnt+1], cnt)
                start_period = str(year_list[cnt]) + "." + month_list[0]
                end_period   = str(year_list[cnt+1]) + "." + month_list[1]
                print(start_period, end_period)

            time.sleep(3)
            search_period_set(driver, start_period, end_period)
            time.sleep(3)
            form_search(driver)
            time.sleep(10)
            get_excel(driver)
            time.sleep(3)


C:\Users\JSSON\AppData\Local\Temp/ipykernel_20016/2788999859.py:16: DeprecationWarning: use driver.switch_to.alert instead
  result = driver.switch_to_alert()


2000.01 2003.12
2004.01 2006.12
2007.01 2009.12
조회 후 엑셀다운로드를 하십시오.
2010.01 2012.12
2013.01 2015.12
2016.01 2018.12
2019.01 2021.12
2000.01 2003.12
2004.01 2006.12
2007.01 2009.12
2010.01 2012.12
2013.01 2015.12
2016.01 2018.12
2019.01 2021.12
2000.01 2003.12
2004.01 2006.12
2007.01 2009.12
2010.01 2012.12
2013.01 2015.12
2016.01 2018.12
2019.01 2021.12
2000.01 2003.12
2004.01 2006.12
2007.01 2009.12
2010.01 2012.12
2013.01 2015.12
2016.01 2018.12
2019.01 2021.12
2000.01 2003.12
2004.01 2006.12
2007.01 2009.12
2010.01 2012.12
2013.01 2015.12
2016.01 2018.12
2019.01 2021.12
2000.01 2003.12
2004.01 2006.12
2007.01 2009.12
2010.01 2012.12
2013.01 2015.12
2016.01 2018.12
2019.01 2021.12
2000.01 2003.12
2004.01 2006.12
2007.01 2009.12
2010.01 2012.12
2013.01 2015.12
2016.01 2018.12
2019.01 2021.12
2000.01 2003.12
2004.01 2006.12
2007.01 2009.12
2010.01 2012.12
2013.01 2015.12
2016.01 2018.12
2019.01 2021.12


In [85]:
import os
os.getcwd()
dir_path = "../data/수출입무역통계/" # 파일위치 폴더 
fd_filenm = os.listdir(dir_path) # 모든 파일명 리스트를 가져옴
fd_filenm=[f_nm for f_nm in fd_filenm if f_nm[-3:]=='xls']
fd_filenm

['품목별 국가별  수출입실적_202110293 (1).xls',
 '품목별 국가별  수출입실적_202110293 (10).xls',
 '품목별 국가별  수출입실적_202110293 (100).xls',
 '품목별 국가별  수출입실적_202110293 (11).xls',
 '품목별 국가별  수출입실적_202110293 (12).xls',
 '품목별 국가별  수출입실적_202110293 (13).xls',
 '품목별 국가별  수출입실적_202110293 (14).xls',
 '품목별 국가별  수출입실적_202110293 (15).xls',
 '품목별 국가별  수출입실적_202110293 (16).xls',
 '품목별 국가별  수출입실적_202110293 (17).xls',
 '품목별 국가별  수출입실적_202110293 (18).xls',
 '품목별 국가별  수출입실적_202110293 (19).xls',
 '품목별 국가별  수출입실적_202110293 (2).xls',
 '품목별 국가별  수출입실적_202110293 (20).xls',
 '품목별 국가별  수출입실적_202110293 (21).xls',
 '품목별 국가별  수출입실적_202110293 (22).xls',
 '품목별 국가별  수출입실적_202110293 (23).xls',
 '품목별 국가별  수출입실적_202110293 (24).xls',
 '품목별 국가별  수출입실적_202110293 (25).xls',
 '품목별 국가별  수출입실적_202110293 (26).xls',
 '품목별 국가별  수출입실적_202110293 (27).xls',
 '품목별 국가별  수출입실적_202110293 (28).xls',
 '품목별 국가별  수출입실적_202110293 (29).xls',
 '품목별 국가별  수출입실적_202110293 (3).xls',
 '품목별 국가별  수출입실적_202110293 (30).xls',
 '품목별 국가별  수출입실적_202110293 (31).xls',
 '품목별 국가별  수출입

In [95]:
def concat_read_file(arg_fnm_list):
    global dir_path
    fd_path = dir_path
    flag = True

    dfs = pd.DataFrame()
    for fnm in arg_fnm_list:
        df = pd.read_excel(fd_path+fnm, skiprows=[0, 1, 2, 3, 5], thousands=',') # 파일을 열었을때 무시하는 행            
        if flag : 
            dfs = df
            flag = False
        else:
            temp = pd.concat([dfs,df], join='outer', ignore_index=True, axis=0)
            dfs = temp
    return dfs 

In [96]:
df=concat_read_file(fd_filenm)

C:\Users\JSSON\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [97]:
df

,기간,품목명,품목코드,국가명,수출중량,수입중량,수출금액,수입금액,무역수지
0,2004.01,양파,703101000,괌,0.1,0.0,0,0,0
1,2004.01,양파,703101000,캐나다,0.0,0.0,1,0,1
2,2004.01,양파,703101000,중국,0.0,9256.1,0,1930,-1930
3,2004.01,양파,703101000,미국,0.5,0.0,2,0,2
4,2004.02,양파,703101000,싱가포르,0.1,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...
17549,2003.11,양파,703101000,미국,1.3,0.0,5,0,5
17550,2003.11,양파,703101000,중국,0.0,11628.0,0,1452,-1452
17551,2003.12,양파,703101000,미국,1.3,20.4,4,14,-10
17552,2003.12,양파,703101000,캐나다,0.1,0.0,1,0,1


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17554 entries, 0 to 17553
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기간      17554 non-null  float64
 1   품목명     17554 non-null  object 
 2   품목코드    17554 non-null  object 
 3   국가명     17554 non-null  object 
 4   수출중량    17554 non-null  float64
 5   수입중량    17554 non-null  float64
 6   수출금액    17554 non-null  object 
 7   수입금액    17554 non-null  object 
 8   무역수지    17554 non-null  object 
dtypes: float64(3), object(6)
memory usage: 1.2+ MB
